In [1]:
import pandas as pd
import numpy as np

seed = 18022004
np.random.seed(seed)

In [55]:
data_prefix: str = 'data'
repo_prefix: str = f'{data_prefix}/repos'

data_name: str = 'first_dataset.parquet'

repo_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{data_name}', engine = 'fastparquet')

In [56]:
repo_df

,id,fromLib,toLib,repoName,repoOwner,repoSplitName,startCommit,endCommit,fileName,startCode,endCode,diff,startCommitChanges,endCommitChanges
0,0,junit:junit,org.junit.jupiter:junit-jupiter-params,svenkubiak_mangooio,svenkubiak,mangooio,3a6b91fa396307230aba6394ceb70ccf52cd4184,3a6b91fa396307230aba6394ceb70ccf52cd4184,mangooio-benchmarks/pom.xml,"<project xmlns=""http://maven.apache.org/POM/4....","<project xmlns=""http://maven.apache.org/POM/4....",diff --git a/mangooio-benchmarks/pom.xml b/man...,+org.junit.jupiter:junit-jupiter-params\n+org....,+org.junit.jupiter:junit-jupiter-params\n+org....
1,1,junit:junit,org.junit.jupiter:junit-jupiter-params,svenkubiak_mangooio,svenkubiak,mangooio,3a6b91fa396307230aba6394ceb70ccf52cd4184,3a6b91fa396307230aba6394ceb70ccf52cd4184,mangooio-benchmarks/src/main/java/Main.java,\n\nimport io.mangoo.core.Application;\nimport...,,diff --git a/mangooio-benchmarks/src/main/java...,+org.junit.jupiter:junit-jupiter-params\n+org....,+org.junit.jupiter:junit-jupiter-params\n+org....
2,2,junit:junit,org.junit.jupiter:junit-jupiter-params,svenkubiak_mangooio,svenkubiak,mangooio,3a6b91fa396307230aba6394ceb70ccf52cd4184,3a6b91fa396307230aba6394ceb70ccf52cd4184,mangooio-benchmarks/src/test/java/mangooio/Tes...,,package mangooio;\n\nimport de.svenkubiak.embe...,diff --git a/mangooio-benchmarks/src/test/java...,+org.junit.jupiter:junit-jupiter-params\n+org....,+org.junit.jupiter:junit-jupiter-params\n+org....
3,3,junit:junit,org.junit.jupiter:junit-jupiter-params,svenkubiak_mangooio,svenkubiak,mangooio,3a6b91fa396307230aba6394ceb70ccf52cd4184,3a6b91fa396307230aba6394ceb70ccf52cd4184,mangooio-benchmarks/src/test/java/mangooio/Tes...,package mangooio;\n\nimport org.junit.BeforeCl...,,diff --git a/mangooio-benchmarks/src/test/java...,+org.junit.jupiter:junit-jupiter-params\n+org....,+org.junit.jupiter:junit-jupiter-params\n+org....
4,4,junit:junit,org.junit.jupiter:junit-jupiter-params,svenkubiak_mangooio,svenkubiak,mangooio,3a6b91fa396307230aba6394ceb70ccf52cd4184,3a6b91fa396307230aba6394ceb70ccf52cd4184,mangooio-benchmarks/src/test/java/mangooio/con...,package mangooio.controllers;\n\nimport static...,package mangooio.controllers;\n\nimport static...,diff --git a/mangooio-benchmarks/src/test/java...,+org.junit.jupiter:junit-jupiter-params\n+org....,+org.junit.jupiter:junit-jupiter-params\n+org....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14939,14939,junit:junit,org.junit.platform:junit-platform-surefire-pro...,h908714124_jbock,h908714124,jbock,497014fb5dc17b4cb804e33b0a61a4f913b8e2b0,497014fb5dc17b4cb804e33b0a61a4f913b8e2b0,examples/src/test/java/net/jbock/examples/RmAr...,package net.jbock.examples;\n\nimport static j...,package net.jbock.examples;\n\nimport static j...,diff --git a/examples/src/test/java/net/jbock/...,+org.junit.jupiter:junit-jupiter-engine\n+org....,+org.junit.jupiter:junit-jupiter-engine\n+org....
14940,14940,junit:junit,org.junit.platform:junit-platform-surefire-pro...,h908714124_jbock,h908714124,jbock,497014fb5dc17b4cb804e33b0a61a4f913b8e2b0,497014fb5dc17b4cb804e33b0a61a4f913b8e2b0,examples/src/test/java/net/jbock/examples/Simp...,package net.jbock.examples;\n\nimport net.jboc...,package net.jbock.examples;\n\nimport net.jboc...,diff --git a/examples/src/test/java/net/jbock/...,+org.junit.jupiter:junit-jupiter-engine\n+org....,+org.junit.jupiter:junit-jupiter-engine\n+org....
14941,14941,junit:junit,org.junit.platform:junit-platform-surefire-pro...,h908714124_jbock,h908714124,jbock,497014fb5dc17b4cb804e33b0a61a4f913b8e2b0,497014fb5dc17b4cb804e33b0a61a4f913b8e2b0,examples/src/test/java/net/jbock/examples/TarA...,package net.jbock.examples;\n\nimport net.jboc...,package net.jbock.examples;\n\nimport net.jboc...,diff --git a/examples/src/test/java/net/jbock/...,+org.junit.jupiter:junit-jupiter-engine\n+org....,+org.junit.jupiter:junit-jupiter-engine\n+org....
14942,14942,junit:junit,org.junit.platform:junit-platform-surefire-pro...,h908714124_jbock,h908714124,jbock,49701

In [57]:
sampled = repo_df[~repo_df['fileName'].str.contains('pom.xml')]

In [ ]:
print(sampled.iloc[2]['diff'])

In [59]:
sample_cnt = 2000
random_sampled = sampled.sample(n = sample_cnt, random_state = seed)

In [60]:
random_sampled

,id,fromLib,toLib,repoName,repoOwner,repoSplitName,startCommit,endCommit,fileName,startCode,endCode,diff,startCommitChanges,endCommitChanges
13424,13424,log4j:log4j,org.slf4j:slf4j-log4j12,apache_helix,apache,helix,f8a0f320a4ec4fb4d4f107d65131d903a09a297a,f8a0f320a4ec4fb4d4f107d65131d903a09a297a,helix-core/src/main/java/org/apache/helix/cont...,package org.apache.helix.controller.stages;\n\...,package org.apache.helix.controller.stages;\n\...,diff --git a/helix-core/src/main/java/org/apac...,+org.slf4j:slf4j-api\n+org.slf4j:slf4j-log4j12...,+org.slf4j:slf4j-api\n+org.slf4j:slf4j-log4j12...
12974,12974,com.google.code.gson:gson,com.fasterxml.jackson.core:jackson-core,itesla_ipst,itesla,ipst,5b4d360f0f55d7f37d67931154758212f79825c7,5b4d360f0f55d7f37d67931154758212f79825c7,online-server/src/main/java/eu/itesla_project/...,"/**\n * Copyright (c) 2016, All partners of th...","/**\n * Copyright (c) 2016, All partners of th...",diff --git a/online-server/src/main/java/eu/it...,+com.fasterxml.jackson.core:jackson-core\n-com...,+com.fasterxml.jackson.core:jackson-core\n-com...
4185,4185,org.springframework:spring-web,org.springframework.boot:spring-boot-starter-web,ralscha_spring4ws-demos,ralscha,spring4ws-demos,a41ba31a897230434ea564a91ddb2b04ad25157e,a41ba31a897230434ea564a91ddb2b04ad25157e,src/main/webapp/griddemo/old/WampProxy.js,"Ext.define('AB.data.proxy.WampProxy', {\n\text...",,diff --git a/src/main/webapp/griddemo/old/Wamp...,+org.springframework.boot:spring-boot-starter-...,+org.springframework.boot:spring-boot-starter-...
11352,11352,org.slf4j:slf4j-log4j12,org.apache.logging.log4j:log4j-api,Graylog2_graylog2-server,Graylog2,graylog2-server,47ba21f7659e6b72b9573687e5b29f90e0992351,47ba21f7659e6b72b9573687e5b29f90e0992351,graylog2-server/src/main/java/org/graylog2/EST...,/**\n * This file is part of Graylog.\n *\n * ...,/**\n * This file is part of Graylog.\n *\n * ...,diff --git a/graylog2-server/src/main/java/org...,+org.apache.zookeeper:zookeeper\n+org.apache.l...,+org.apache.zookeeper:zookeeper\n+org.apache.l...
14757,14757,log4j:log4j,ch.qos.logback:logback-classic,joliciel-informatique_talismane,joliciel-informatique,talismane,1ed8b995e9d677dff7eee752a36c7b43923360c1,1ed8b995e9d677dff7eee752a36c7b43923360c1,talismane_core/src/main/java/com/joliciel/tali...,package com.joliciel.talismane.parser;\n\nimpo...,package com.joliciel.talismane.parser;\n\nimpo...,diff --git a/talismane_core/src/main/java/com/...,+org.slf4j:slf4j-api\n+ch.qos.logback:logback-...,+org.slf4j:slf4j-api\n+ch.qos.logback:logback-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2139,2139,commons-httpclient:commons-httpclient,org.apache.httpcomponents:httpmime,fcrepo3_fcrepo,fcrepo3,fcrepo,c2bdb5f59713dd69efe1e10b84f57560753c6270,c2bdb5f59713dd69efe1e10b84f57560753c6270,fcrepo-integrationtest/fcrepo-integrationtest-...,/* The contents of this file are subject to th...,/* The contents of this file are subject to th...,diff --git a/fcrepo-integrationtest/fcrepo-int...,+org.apache.httpcomponents:httpmime\n+org.apac...,+org.apache.httpcomponents:httpmime\n+org.apac...
9503,9503,org.hibernate.javax.persistence:hibernate-jpa-...,javax.persistence:javax.persistence-api,flowable_flowable-engine,flowable,flowable-engine,ea42e0e9e6dfb8053cb5a69a54cf01ec19364832,ea42e0e9e6dfb8053cb5a69a54cf01ec19364832,modules/activiti-rest/src/main/java/org/activi...,"/* Licensed under the Apache License, Version ...","/* Licensed under the Apache License, Version ...",diff --git a/modules/activiti-rest/src/main/ja...,+javax.persistence:javax.persistence-api\n-org...,+javax.persistence:javax.persistence-api\n-org...
8186,8186,junit:junit,org.junit.platform:junit-platform-launcher,fabric8io_kubernetes-client,fabric8io,kubernetes-client,ff3c6568398a5dfc881121a20aa190133b588d70,ff3c6568398a5dfc881121a20aa190133b588d70,kubernetes-tests/src/test/java/io/fabric8/open...,"/**\n * Copyright (C) 2015 Red Hat, Inc.\n *\n...","/**\n * Copyright (C) 2015 Red Hat, Inc.\n *\n...

In [62]:
random_sampled.to_csv(f'{data_prefix}/{sample_cnt}_samples_dataset.csv', escapechar = '\\')